In [ ]:
# 📦 1. Imports
import kagglehub
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.metrics import classification_report
import numpy as np

# 📂 2. Download Dataset
path = kagglehub.dataset_download("prashant268/chest-xray-covid19-pneumonia")
dataset = Path(path)
train_dir = dataset / "Data/train"
test_dir  = dataset / "Data/test"

# 📸 3. Preprocessing (Friend-style: VGG16 preprocess)
from tensorflow.keras.applications.vgg16 import preprocess_input

train_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_batches = train_datagen.flow_from_directory(
    directory=train_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=True,
    class_mode='categorical'
)

test_batches = test_datagen.flow_from_directory(
    directory=test_dir,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False,
    class_mode='categorical'
)

# 🧠 4. Model Definition (Friend-style, deep CNN)
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(train_batches.num_classes, activation='softmax')
])

# 🧪 5. Compile Model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# 🛠 6. Callbacks (basic)
callbacks = [
    ModelCheckpoint('best_model.keras', save_best_only=True, monitor='loss'),
    ReduceLROnPlateau(monitor='loss', factor=0.3, patience=3, min_lr=1e-6, verbose=1),
    EarlyStopping(monitor='loss', patience=10, restore_best_weights=True, verbose=1)
]

# 🚀 7. Train Model
history = model.fit(
    train_batches,
    epochs=10,
    steps_per_epoch=len(train_batches),
    callbacks=callbacks
)

# ✅ 8. Evaluate Model
loss, acc = model.evaluate(test_batches, steps=len(test_batches))
print(f"\n✅ Test Loss: {loss:.4f}")
print(f"✅ Test Accuracy: {acc:.4f}")

# 📊 9. Classification Report
predictions = model.predict(test_batches, steps=len(test_batches))
predicted_classes = np.argmax(predictions, axis=1)
true_classes = test_batches.classes
labels = list(test_batches.class_indices.keys())

print("\n📄 Classification Report:\n")
print(classification_report(true_classes, predicted_classes, target_names=labels))


100%|██████████| 2.06G/2.06G [00:14<00:00, 149MB/s]

Extracting files...


Found 5144 images belonging to 3 classes.
Found 1288 images belonging to 3 classes.


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 72s 395ms/step - accuracy: 0.7073 - loss: 6.5493 - learning_rate: 1.0000e-04
Epoch 2/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 71s 367ms/step - accuracy: 0.8973 - loss: 0.3004 - learning_rate: 1.0000e-04
Epoch 3/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 59s 367ms/step - accuracy: 0.9267 - loss: 0.2247 - learning_rate: 1.0000e-04
Epoch 4/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 62s 384ms/step - accuracy: 0.9413 - loss: 0.1689 - learning_rate: 1.0000e-04
Epoch 5/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 63s 394ms/step - accuracy: 0.9541 - loss: 0.1414 - learning_rate: 1.0000e-04
Epoch 6/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 63s 394ms/step - accuracy: 0.9626 - loss: 0.1209 - learning_rate: 1.0000e-04
Epoch 7/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 61s 379ms/step - accuracy: 0.9671 - loss: 0.1063 - learning_rate: 1.0000e-04
Epoch 8/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 60s 372ms/step - accuracy: 0.9734 - loss: 0.0725 - learning_rate: 1.0000e-04
Epoch 9/10
161/161 ━━━━━━━━━━━━━━━━━━━━ 79s 355ms/step - accurac

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Reuse variables: predicted_classes, true_classes, labels

# Calculate & print accuracy
tta_accuracy = accuracy_score(true_classes, predicted_classes)
print(f"\n✅ Final Accuracy: {tta_accuracy:.4f}")

# Confusion Matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)
print("\n🔍 Confusion Matrix:")
print(conf_matrix)

# Classification Report (detailed dict)
report = classification_report(true_classes, predicted_classes, target_names=labels, output_dict=True)

# Print class-wise stats cleanly
print("\n📊 Class-wise Performance:")
for label in labels:
    cls = report[label]
    print(f"  {label.upper()} — Precision: {cls['precision']:.4f}, Recall: {cls['recall']:.4f}, F1: {cls['f1-score']:.4f}")



✅ Final Accuracy: 0.9596

🔍 Confusion Matrix:
[[113   3   0]
 [  0 299  18]
 [  4  27 824]]

📊 Class-wise Performance:
  COVID19 — Precision: 0.9658, Recall: 0.9741, F1: 0.9700
  NORMAL — Precision: 0.9088, Recall: 0.9432, F1: 0.9257
  PNEUMONIA — Precision: 0.9786, Recall: 0.9637, F1: 0.9711


In [ ]:
model.save("chest_cnn_final_96acc.h5")